In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import glob

train_jsons = sorted(glob.glob('/content/drive/MyDrive/스마트팩토리_배터리/data/resized_data/train/label/*.json'))
len(train_jsons)

0

In [ ]:
import os
import json

# JSON 파일이 저장된 디렉토리 경로
json_dir = '/content/drive/MyDrive/smartFactory/JSON'
# YOLOv8 형식의 라벨 파일이 저장될 디렉토리 경로
labels_dir = '/content/drive/MyDrive/smartFactory/Text'

# 디렉토리가 존재하지 않으면 생성
os.makedirs(labels_dir, exist_ok=True)

# 클래스 이름과 ID 매핑 (예시: Pollution - 0, Damaged - 1)
class_name_to_id = {'Pollution': 2, 'Damaged': 1}

def cal_relatice_position(xmin,ymin,xmax,ymax,image_width,image_height):
    center_x=(xmin+0.5*(xmax-xmin)) / image_width
    center_y=(ymin+0.5*(ymax-ymin)) / image_height
    width = (xmax-xmin) / image_width
    height = (ymax-ymin) / image_height
    print(center_x,center_y,width,height)
    return [round(center_x,6),round(center_y,6),round(width,6),round(height,6)]

def process_json_file(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    image_info = data['image_info']
    defect_boxs = data['defect_boxs']
    outline_box=data['outline_box'] #리스트 [x1,y1,x2,y2]
    outline_box=cal_relatice_position(outline_box[0],outline_box[1],outline_box[2],outline_box[3],image_info['width'],image_info['height'])

    image_width = image_info['width']
    image_height = image_info['height']
    image_filename = image_info['file_name']
    image_id = image_info['id']

    # YOLOv8 형식의 라벨 파일 경로
    label_file_path = os.path.join(labels_dir, f'{os.path.splitext(image_filename)[0]}.txt')

    with open(label_file_path, 'w') as label_file:
        label_file.write(f'{0} {outline_box[0]} {outline_box[1]} {outline_box[2]} {outline_box[3]}\n')
        for defect in defect_boxs:
            class_id = class_name_to_id[defect['name']]
            box = defect['box']
            # YOLO 형식으로 변환
            x_center = (box[0] + box[2]) / 2 / image_width
            y_center = (box[1] + box[3]) / 2 / image_height
            width = (box[2] - box[0]) / image_width
            height = (box[3] - box[1]) / image_height
            label_file.write(f'{class_id} {round(x_center,6)} {round(y_center,6)} {round(width,6)} {round(height,6)}\n')

# 모든 JSON 파일 처리
for json_file in os.listdir(json_dir):
    if json_file.endswith('.json'):
        process_json_file(os.path.join(json_dir, json_file))
'''
# data.yaml 파일 생성
data_yaml_content = f"""
train: ../train/images
val: ../valid/images
test: ../test/images

nc: {len(class_name_to_id)}
names: {list(class_name_to_id.keys())}
"""

with open('data.yaml', 'w') as data_yaml_file:
    data_yaml_file.write(data_yaml_content)'''

print("YOLOv8 데이터셋 준비가 완료되었습니다.")

0.5 0.5166666666666667 0.34074074074074073 0.9444444444444444
0.5 0.512962962962963 0.3925925925925926 0.9666666666666667
0.4981481481481482 0.5222222222222223 0.40370370370370373 0.9481481481481482
0.4981481481481482 0.5203703703703704 0.2777777777777778 0.7888888888888889
0.4981481481481482 0.5222222222222223 0.3888888888888889 0.9481481481481482
0.4981481481481482 0.5203703703703704 0.3592592592592593 0.9518518518518518
0.4981481481481482 0.512962962962963 0.3888888888888889 0.9666666666666667
0.5 0.524074074074074 0.4074074074074074 0.9444444444444444
0.5 0.5111111111111111 0.2962962962962963 0.725925925925926
0.4981481481481482 0.5148148148148148 0.337037037037037 0.9407407407407408
0.5 0.5148148148148148 0.4 0.9629629629629629
0.4981481481481482 0.5203703703703704 0.36666666666666664 0.9518518518518518
0.4981481481481482 0.5166666666666667 0.337037037037037 0.9444444444444444
0.5 0.5203703703703704 0.4740740740740741 0.9518518518518518
0.5 0.5222222222222223 0.4740740740740741 0.

In [2]:
pip install scikit-learn

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 데이터셋 경로 설정
image_dir = '/content/drive/MyDrive/스마트팩토리_배터리/data/최종_라벨링/이미지'
label_dir = '/content/drive/MyDrive/스마트팩토리_배터리/data/최종_라벨링/라벨링'

# 이미지와 라벨 파일 리스트 얻기
images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

# train, val, test로 분할 (70%, 20%, 10%)
train_images, temp_images, train_labels, temp_labels = train_test_split(images, labels, test_size=0.3, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=0.33, random_state=42)

# 분할된 데이터를 적절한 폴더로 이동
def move_files(file_list, source_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    for file in file_list:
        shutil.move(os.path.join(source_dir, file), os.path.join(dest_dir, file))

move_files(train_images, image_dir, '/content/drive/MyDrive/YOLO_data/train/images')
move_files(val_images, image_dir, '/content/drive/MyDrive/YOLO_data/vaild/images')
move_files(test_images, image_dir, '/content/drive/MyDrive/YOLO_data/test/labels')

move_files(train_labels, label_dir, '/content/drive/MyDrive/YOLO_data/train/labels')
move_files(val_labels, label_dir, '/content/drive/MyDrive/YOLO_data/vaild/labels')
move_files(test_labels, label_dir, '/content/drive/MyDrive/YOLO_data/test/labels')

In [4]:
pip install pyyaml

In [5]:
#yaml파일 생성
import yaml

# yaml 파일 데이터 정의
data = {
    'train': '/content/drive/MyDrive/YOLO_data/train/images',
    'val': '/content/drive/MyDrive/YOLO_data/vaild/images',
    'test': '/content/drive/MyDrive/YOLO_data/test/images',
    'nc': 3,  # 클래스 수
    'names': ['Battery','Pollution', 'Damaged']  # 클래스 이름 리스트
}

# yaml 파일 생성
with open('/content/drive/MyDrive/YOLO_data/data.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("data.yaml 파일이 생성되었습니다.")

data.yaml 파일이 생성되었습니다.


In [6]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.7/802.7 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [7]:
from ultralytics import YOLO
import yaml
# 2. YOLOv8 모델 학습
# 모델 로드 (사전 학습된 모델 사용)
model = YOLO('/content/drive/MyDrive/스마트팩토리_배터리/model/1.모델/yolov8n.pt')  # YOLOv8의 다양한 버전 중 하나 선택 (예: yolov8s.pt, yolov8m.pt 등)

# 데이터셋 설정 파일 경로
data_config = '/content/drive/MyDrive/YOLO_data/data.yaml'

# 모델 학습
model.train(data=data_config, epochs=50, imgsz=270)

print("모델 학습이 완료되었습니다.")


Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/스마트팩토리_배터리/model/1.모델/yolov8n.pt, data=/content/drive/MyDrive/YOLO_data/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=270, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False,

100%|██████████| 755k/755k [00:00<00:00, 93.3MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 69.7MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅
WARNING ⚠️ imgsz=[270] must be multiple of max stride 32, updating to [288]


train: Scanning /content/drive/MyDrive/YOLO_data/train/labels... 702 images, 0 backgrounds, 0 corrupt:  67%|██████▋   | 702/1043 [08:27<04:06,  1.38it/s]


KeyboardInterrupt: 

In [ ]:
import os
# 3. 모델 평가 및 테스트
# 학습된 모델의 정확한 경로를 확인
model_dir = 'runs/detect/train2/weights'
model_path = os.path.join(model_dir, 'best.pt')

# 모델 경로 확인
if os.path.exists(model_path):
    model = YOLO(model_path)
    # 모델 평가 (검증 데이터셋 사용)
    results = model.val(data=data_config)
    print(results)
else:
    print(f"모델 파일을 찾을 수 없습니다: {model_path}")

Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/smartFactory/YOLO_data/vaild/labels.cache... 54 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54/54 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


                   all         54        762       0.76      0.491      0.516      0.365
               Battery         54         54      0.982          1      0.995      0.793
             Pollution         54        212      0.693      0.181      0.224      0.137
               Damaged         54        496      0.604      0.292      0.328      0.165
Speed: 0.2ms preprocess, 7.7ms inference, 0.0ms loss, 7.5ms postprocess per image
Results saved to runs/detect/val5
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x780ece2fda50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0

In [ ]:
import os
# 3. 모델 평가 및 테스트
# 학습된 모델의 정확한 경로를 확인
model_dir = 'runs/detect/train2/weights'
model_path = os.path.join(model_dir, 'best.pt')

# 모델 경로 확인
if os.path.exists(model_path):
    model = YOLO(model_path)
    # 모델 평가 (검증 데이터셋 사용)
    results = model.val(data=data_config)
    print(results)
else:
    print(f"모델 파일을 찾을 수 없습니다: {model_path}")

test_dir='/content/drive/MyDrive/smartFactory/YOLO_data/test/images'
images = [f for f in os.listdir(test_dir) if f.endswith('.png')]
for image in images:
    image_path = os.path.join(test_dir, image)
    results = model(image_path)
     # 예측 결과를 개별적으로 저장
    for idx, result in enumerate(results):
        save_path = f'/content/drive/MyDrive/smartFactory/YOLO_data/test/predictions/{os.path.splitext(image)[0]}_prediction_{idx}.png'
        result.save(filename=save_path)


Ultralytics YOLOv8.2.57 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/smartFactory/YOLO_data/vaild/labels.cache... 54 images, 0 backgrounds, 0 corrupt: 100%|██████████| 54/54 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.15s/it]


                   all         54        762       0.76      0.491      0.516      0.365
               Battery         54         54      0.982          1      0.995      0.793
             Pollution         54        212      0.693      0.181      0.224      0.137
               Damaged         54        496      0.604      0.292      0.328      0.165
Speed: 0.1ms preprocess, 3.2ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs/detect/val6
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x780ed63d60b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0

In [ ]:
import shutil

# run 폴더를 Google 드라이브로 복사
source = '/content/runs'
destination = '/content/drive/MyDrive/smartFactory/YOLO_data/runs'

shutil.copytree(source, destination)

'/content/drive/MyDrive/smartFactory/YOLO_data/runs'